In [11]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

In [17]:
z = pd.DataFrame({'news': ["Alia Bhatt, Aditya Roy Kapur and Sanjay Dutt starrer 'Sadak 2' trailer has become the third most disliked YouTube video in the world, amid nepotism debate. So far, it has received 10 million dislikes and is behind Justin Bieber's song 'Baby' (11 million dislikes) and YouTube Rewind 2018 (18 million dislikes). The trailer of the film was released on Wednesday.","Following MS Dhoni's retirement from international cricket, actor Ranveer Singh took to Instagram and shared throwback pictures of himself with Dhoni and revealed he took an ad job when he was around 22 years old only because it featured Dhoni.he wrote."] })
z

,news
0,"Alia Bhatt, Aditya Roy Kapur and Sanjay Dutt s..."
1,Following MS Dhoni's retirement from internati...


In [18]:
def pre_process(text):
    text = text.lower()
    text = re.sub('&lt;/?.*?&gt', '&lt;&gt', text)
    text = re.sub("(\\d|\\W)+", " ", text)
    return text

In [19]:
z['news'] = z['news'].apply(lambda x: pre_process(x))

In [21]:
docs = z['news'].tolist()
docs

['alia bhatt aditya roy kapur and sanjay dutt starrer sadak trailer has become the third most disliked youtube video in the world amid nepotism debate so far it has received million dislikes and is behind justin bieber s song baby million dislikes and youtube rewind million dislikes the trailer of the film was released on wednesday ',
 'following ms dhoni s retirement from international cricket actor ranveer singh took to instagram and shared throwback pictures of himself with dhoni and revealed he took an ad job when he was around years old only because it featured dhoni he wrote ']

In [22]:
#preprocessed_text = pre_process(text)
cv = CountVectorizer(max_df=0.85, stop_words=stopwords, max_features=10000)
word_count_vector = cv.fit_transform(docs)

In [23]:
tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

In [24]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

In [25]:
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    sorted_items = sorted_items[:topn]
    score_vals, feature_vals = [], []
    for idx, score in sorted_items:
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
        
    results = {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]] = score_vals[idx]
    return results

In [27]:
feature_names = cv.get_feature_names()
doc = docs[0]
tf_idf_vector = tfidf_transformer.fit_transform(cv.transform([doc]))
sorted_items = sort_coo(tf_idf_vector.tocoo())
keywords = extract_topn_from_vector(feature_names, sorted_items)
keywords

{'million': 0.408,
 'dislikes': 0.408,
 'youtube': 0.272,
 'trailer': 0.272,
 'world': 0.136,
 'wednesday': 0.136,
 'video': 0.136,
 'third': 0.136,
 'starrer': 0.136,
 'song': 0.136}